In [151]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [152]:
import copy, math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [153]:
import gepard as g
print(g.__version__)

0.9.10


In [154]:
print(g.__file__)

/home/quantum/gepard/src/gepard/__init__.py


In [155]:
pt = g.dset[75][0]

In [156]:
g.describe_data(g.dset[75])

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
35 x XGAMMA  H1      N/A    75  arXiv:0910.5831
----------------------------------------------
TOTAL = 35


In [157]:
pt.observable, pt.val

('XGAMMA', 2156.0)

In [158]:
pt.Q2

3.3

In [159]:
class MyTheory(g.PWNormGPD, g.MellinBarnesCFF, g.MellinBarnesTFF, g.DVMP):
    pass

In [160]:
th0 = MyTheory(p=0)

In [161]:
th0.parameters.update({'a2': 0.123, 'ns': 0.15203977400126656, 'al0s': 1.1575112053849788, 'al0g': 1.2473462505404416, 'alps': 0.5312673554988383, 'ms2': 2.740237296159404, 'secs': 0.04939583198250308, 'this': -0.05411781123920072, 'alpg': 0.9999999988492917, 'mg2': 0.11935246209863909, 'secg': 1.342447132772215, 'thig': -0.9997960402412689})

In [162]:
th0.parameters['a2']

0.123

In [163]:
th0.XGAMMA(pt)

41869.57050702964

In [164]:
th0.tff(xi=pt.xi, t=pt.t, Q2=pt.Q2, meson='rho0')

array([-1277.35815592, -3191.92735482,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ])

In [165]:
th0.ImH_V(pt)

-3191.9273548168135

In [166]:
#gamma0 = g.adim.non_singlet_LO(2,th0.nf)
#beta0 = g.qcd.beta(th0.p,th0.nf)

#abc = g.wilson.calc_wce(th0, pt.Q2, 'DVMP', 2)
#print(gamma0,beta0,abc)


We reproduce this number step by step, as done internally by Gepard.

In [167]:
wce_ar_dvmp = g.wilson.calc_wce(th0, pt.Q2, 'DVMP', 0)

In [168]:
wce_ar_dvmp2 = g.wilson.calc_wce(th0, pt.Q2, 'DVMP', 2)

In [169]:
wce_ar_dvmp[0, 0, :]

array([1.19661056+0.00022386j, 2.11518273+0.0003092j ,
       5.38462485+0.00055892j])

In [170]:
wce_ar_dvmp2[0, 0, :]

array([1.09324942+0.00020453j, 1.93247693+0.00028249j,
       4.91951032+0.00051064j])

In [171]:
h_prerot = th0.H(pt.xi, pt.t)

In [172]:
h_prerot.shape    #  MBpoints  x  flavor basis (sea quarks, G, u_val, d_val)

(96, 4)

In [173]:
h_prerot[0,:]

array([2.49083853-0.00362085j, 6.56688237-0.01322384j,
       0.        +0.j        , 0.        +0.j        ])

In [174]:
h = np.einsum('fa,ja->jf', th0.frot_rho0_4, h_prerot)    # GPD in evolution basis

In [175]:
wce_ar_dvmp.shape

(3, 96, 3)

In [176]:
reh_aux0, imh_aux0 = th0._mellin_barnes_integral(pt.xi, wce_ar_dvmp, h)
reh_aux2, imh_aux2 = th0._mellin_barnes_integral(pt.xi, wce_ar_dvmp2, h)
reh_aux2 *=th0.parameters['a2']
imh_aux2 *=th0.parameters['a2']
reh_aux = reh_aux0 + reh_aux2
imh_aux = imh_aux0 + imh_aux2

In [177]:
reh_aux, imh_aux

(-70206.7025918574, -175436.0697162418)

In [178]:
astrong = 2 * np.pi * g.qcd.as2pf(th0.p, th0.nf,  pt.Q2, th0.asp[th0.p], th0.r20)

In [179]:
reh, imh = (g.constants.CF * g.constants.F_rho0 * astrong / g.constants.NC / np.sqrt(pt.Q2)
           ) * np.array([reh_aux, imh_aux])

In [180]:
imh

-3191.9273548168135